In [134]:
# import all libraries you need here
import pandas as pd
import numpy as np
import pathlib as pl

import scanpy as sc
import os

# Step 0: Download the data

In [135]:
bulk_train_x = pd.read_csv("train_data/train_bulk.csv", index_col=0)
bulk_train_y = pd.read_csv("train_data/train_bulk_trueprops.csv", index_col=0)
bulk_test_x = pd.read_csv("test_data/test_bulk.csv", index_col=0)

In [136]:
sc_train_x = sc.read_h5ad("train_data/train_adata.h5ad")
sc_test = sc.read_h5ad("test_data/test_adata.h5ad")

In [137]:
print(sc_train_x.var.shape) # gene level
sc_train_x.var.head()

(7725, 6)


,n_cells,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
Unnamed: 0,,,,,,
NOC2L,6735,False,6735,0.143681,87.143757,7527.0
HES4,8287,False,8287,0.330635,84.181190,17321.0
ISG15,19640,False,19640,0.871934,62.509783,45678.0
TNFRSF18,12021,False,12021,0.682345,77.053467,35746.0
TNFRSF4,7880,False,7880,0.484510,84.958100,25382.0


In [138]:
print(sc_train_x.obs.shape) # cell level
sc_train_x.obs.head()

(32374, 10)


,Sample,Patient,Tumor status,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,highLevelType,chemo
AAACCCAAGGAGACCT-1_1,s1,P1,Tumor,991,991,2156.0,131.0,6.076067,T,post
AAACCCAGTAGACAGC-1_1,s1,P1,Tumor,1063,1063,2485.0,84.0,3.380282,T,post
AAACCCAGTAGATCGG-1_1,s1,P1,Tumor,855,855,1993.0,87.0,4.365278,T,post
AAACCCAGTATCACCA-1_1,s1,P1,Tumor,1002,1002,2147.0,139.0,6.474150,B,post
AAACCCAGTTGGAGGT-1_1,s1,P1,Tumor,958,958,2438.0,108.0,4.429861,T,post


In [139]:
print(sc_train.X.shape) # expression of the cells by genes

(32374, 7725)


# Step 1: Perform SC clustering 

In [140]:
# Preprocessing
# 1. Normalize total counts per cell (optional but common)
sc.pp.normalize_total(sc_train, target_sum=1e4)

# 2. Log-transform the data
sc.pp.log1p(sc_train)

# 3. Identify highly variable genes (optional but improves PCA)
sc.pp.highly_variable_genes(sc_train, n_top_genes=2000, subset=True)

# 4. Scale the data (mean=0, variance=1)
sc.pp.scale(sc_train)

c:\Users\edonn\miniconda3\Lib\functools.py:934: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)


In [141]:
# PCA
sc.tl.pca(sc_train, svd_solver='arpack')

In [150]:
sc_train.obsm['X_pca'].shape

(32374, 50)

In [142]:
# K-means, giving a very high penalty to clusters with mixed cell types

In [143]:
# assign to each cell type all the cluster centers that are majority that cell type

# Step 2: Deconvolve the data

In [144]:
# for each cell type x, cacluate x_gene_expression = x_cell_percentage * sum(over all clusters assigned to cell type x) (cluster_cell_percentage * cluster_expression_matrix)

# Step 2bis: Predict on the test data

In [145]:
# Use the same clustering technique to cluster the test data 

# Step 3: Save the required files

In [146]:
# pred_props should be a DataFrame containing the estimated cell type proportions for the patients in all_bulkified
# pred_props.columns = ['index','s5_0','s5_1',...,'s10_3','s10_4'] = np.append(["index"],all_bulkified.columns)
# pred_props['index'] = ['T', 'Endothelial', 'Fibroblast', 'Plasmablast', 'B', 'Myofibroblast',
#       'NK', 'Myeloid', 'Mast']

In [147]:
results_path = pl.Path("path/to/your/results")

In [148]:
assert all(pred_props.columns == np.append(["index"],all_bulkified.columns)), "Wrong columns"

NameError: name 'pred_props' is not defined

In [ ]:
assert all(pred_props['index']== ['T', 'Endothelial', 'Fibroblast', 'Plasmablast', 'B', 'Myofibroblast',
       'NK', 'Myeloid', 'Mast']), "Wrong order for cell types"

In [ ]:
assert all(pred_props.drop("index",axis=1).sum().round()==1), "The proportions for a single patient must sum to 1"

In [ ]:
# cluster_labels should be a DataFrame containing the cluster labels for each cell
# cluster_labels.columns = ["index", "cluster"]
# cluster_labels["index"] = test_adata.columns

In [ ]:
assert all(cluster_labels.columns == ["index", "cluster"]), "Wrong columns"

In [ ]:
assert all(cluster_labels["index"] == test_adata.obs_names), "The cell ids are either not all present or not in the right order"

In [ ]:
import zipfile

archive_name = "LastName_FirstName_Project2.zip" # TODO

with zipfile.ZipFile(results_path / archive_name, "x") as zf:
    with zf.open(f"pred_props.csv", "w") as buffer:
        pred_props.to_csv(buffer)
    with zf.open(f"cluster_membership.csv", "w") as buffer:
        cluster_labels.to_csv(buffer)
    zf.close()